<a href="https://colab.research.google.com/github/MitjaGo/BAR/blob/main/ID3TAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ==========================================
# 🟢 Install Dependencies
# ==========================================
!pip install mutagen pytube ipywidgets tabulate

from mutagen.easyid3 import EasyID3
from mutagen.id3 import ID3, APIC, error
from pytube import YouTube
from IPython.display import display
from google.colab import files
from tabulate import tabulate
import ipywidgets as widgets
import requests
import os
import zipfile
from datetime import datetime

# ==========================================
# 🟢 Step 1: Upload MP3 Files (≤ 50)
# ==========================================
print("⬆️ Upload up to 50 MP3 files:")
upload = files.upload()

mp3_files = list(upload.keys())[:50]
print(f"\n✅ Uploaded {len(mp3_files)} file(s).")

# ==========================================
# 🟢 Step 2: Display Existing ID3 Tags
# ==========================================
rows = []
for f in mp3_files:
    try:
        audio = EasyID3(f)
        title = audio.get("title", [""])[0]
        artist = audio.get("artist", [""])[0]
        album = audio.get("album", [""])[0]
    except error:
        title, artist, album = "", "", ""
    rows.append([f, title, artist, album])

print("\n📋 Current ID3 Tag Info:")
print(tabulate(rows, headers=["File", "Title", "Artist", "Album"], tablefmt="grid"))

# ==========================================
# 🟢 Step 3: Editable Widgets
# ==========================================
file_dd = widgets.Dropdown(options=mp3_files, description="🎵 File:")
title_txt = widgets.Text(description="Title:")
artist_txt = widgets.Text(description="Artist:")
album_txt = widgets.Text(description="Album:")
yt_txt = widgets.Text(description="YouTube Link:")
save_btn = widgets.Button(description="💾 Save Changes", button_style='success')
zip_btn = widgets.Button(description="⬇️ Download ZIP of Edited MP3s", button_style='info')
output = widgets.Output()

def load_tags(f):
    """Load ID3 tags safely."""
    try:
        audio = EasyID3(f)
        return {
            "title": audio.get("title", [""])[0],
            "artist": audio.get("artist", [""])[0],
            "album": audio.get("album", [""])[0]
        }
    except error:
        return {"title": "", "artist": "", "album": ""}

def on_file_change(change):
    """Load tags when selecting a file"""
    f = change['new']
    tags = load_tags(f)
    title_txt.value = tags["title"]
    artist_txt.value = tags["artist"]
    album_txt.value = tags["album"]
    yt_txt.value = ""

file_dd.observe(on_file_change, names='value')

def on_save_clicked(b):
    """Save updated tags + optional YouTube thumbnail"""
    f = file_dd.value
    new_title, new_artist, new_album = title_txt.value.strip(), artist_txt.value.strip(), album_txt.value.strip()
    yt_link = yt_txt.value.strip()

    # Load existing tags (don’t overwrite blank ones)
    existing = load_tags(f)
    audio = EasyID3(f)
    if new_title:
        audio['title'] = new_title
    else:
        audio['title'] = existing['title']
    if new_artist:
        audio['artist'] = new_artist
    else:
        audio['artist'] = existing['artist']
    if new_album:
        audio['album'] = new_album
    else:
        audio['album'] = existing['album']
    audio.save()

    msg = f"✅ Updated tags for {f}."

    # Handle YouTube thumbnail
    if yt_link:
        try:
            yt = YouTube(yt_link)
            thumb_url = yt.thumbnail_url
            if not thumb_url:
                raise ValueError("No thumbnail URL found.")
            img_data = requests.get(thumb_url)
            if img_data.status_code != 200:
                raise ValueError(f"Thumbnail fetch failed ({img_data.status_code}).")
            img_data = img_data.content

            audio_id3 = ID3(f)
            audio_id3.delall('APIC')
            audio_id3.add(APIC(
                encoding=3,
                mime='image/jpeg',
                type=3,
                desc='Cover',
                data=img_data
            ))
            audio_id3.save()
            msg += " 🖼️ Added YouTube thumbnail."
        except Exception as e:
            msg += f" ⚠️ Could not fetch thumbnail ({e})."

    with output:
        output.clear_output()
        print(msg)

def on_zip_clicked(b):
    """ZIP all edited MP3 files for download"""
    now = datetime.now().strftime("%Y%m%d_%H%M%S")
    zip_name = f"edited_mp3s_{now}.zip"
    with zipfile.ZipFile(zip_name, "w") as z:
        for f in mp3_files:
            z.write(f)
    files.download(zip_name)

save_btn.on_click(on_save_clicked)
zip_btn.on_click(on_zip_clicked)

display(file_dd, title_txt, artist_txt, album_txt, yt_txt, save_btn, zip_btn, output)


⬆️ Upload up to 50 MP3 files:
